In [100]:
#manipulate dataframes in python
import pandas as pd

#make API calls with python
import requests

import localvars

#allows us to store results of API call cleanly
import json

# List of Years
strYears =["2012", "2013", "2014", "2015", "2016", "2017"]

# Using for loop 
metric = pd.DataFrame() #creates a new dataframe that's empty
for i in strYears: 

    #construct the API call we will use
    baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=tract:*&in=state:47%%20county:125&key=%s" 

    calledAPI = (baseAPI % (i, "B01001_002E", localvars.apiKey))

    #call the API and collect the response
    response = requests.get(calledAPI)

    #load the response into a JSON, ignoring the first element which is just field labels
    formattedResponse = json.loads(response.text)[1:]

    #flip the order of the response from [population, zipcode] -> [zipcode, population]
    formattedResponse = [item[::-1] for item in formattedResponse]

    #store the response in a dataframe
    strColYear = "y_" + i
    
    
    metrictemp = pd.DataFrame(columns=['id', '2', '3', strColYear], data=formattedResponse)
    del metrictemp['2']
    del metrictemp['3']
    if i == "2012":
        metric=metrictemp
        print(calledAPI)
    else: 
        metric = metric.merge(metrictemp, on='id', how='outer')
    
    
#save that dataframe to a CSV spreadsheet
metric.to_csv('laZipPopulations.csv', index=False)


https://api.census.gov/data/2012/acs/acs5?get=B01001_002E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
